In [1]:
# Packet vs Circuit switching

from math import comb
from functools import reduce

link = 1000         # kb/s
each_user = 100     # kb/s when user is active
active_rate = 0.1   # 10% of the time
num_user = 35       # Number of trials
num_succ = 10       # Number of successes (x)

def prob(n, k, p):
    # n is the number of trials, k is the number of successes, p is the probability of success
    return comb(n, k) * p**k * (1-p)**(n-k)

print(f"Circuit-switching:\n\tcan support {link/each_user} users")
print("Packet-switching:")
print(f"\twith {num_user} users, probability = {num_succ} user active at same time is {prob(num_user, num_succ, active_rate):.10f}")
print(f"\twith {num_user} users, probability > {num_succ} user active at same time is less than {1 - sum((prob(num_user, i, active_rate) for i in range(num_succ+1))):.10f}")
print(f"\twith {num_user} users, probability < {num_succ} user active at same time is less than {sum((prob(num_user, i, active_rate) for i in range(num_succ))):.10f}")
print(f"\twith {num_user} users, probability >= {num_succ} user active at same time is less than {1-sum((prob(num_user, i, active_rate) for i in range(num_succ))):.10f}")
print(f"\twith {num_user} users, probability <= {num_succ} user active at same time is less than {sum((prob(num_user, i, active_rate) for i in range(num_succ+1))):.10f}")

Circuit-switching:
	can support 10.0 users
Packet-switching:
	with 35 users, probability = 10 user active at same time is 0.0013179128
	with 35 users, probability > 10 user active at same time is less than 0.0004242976
	with 35 users, probability < 10 user active at same time is less than 0.9982577896
	with 35 users, probability >= 10 user active at same time is less than 0.0017422104
	with 35 users, probability <= 10 user active at same time is less than 0.9995757024


In [2]:
# Calculate the end-to-end delay

packets_length = 10000              # bits
link_between = 5000                 # m
num_routers = 4                     # between source and destination
processing_time = 10                # msec
transmission_rate = 10**6           # bit/s
propagation_speed = 2.5 * 10**8     # m/s
queue_delay = 0

print("hop_delay = proc + trans + prop + queue")
print(f"{processing_time} msec + {(link_between/propagation_speed) * 1000} msec + {(packets_length/transmission_rate) * 1000} msec + {queue_delay} msec")
hop_delay = processing_time + ((link_between/propagation_speed) * 1000) + (packets_length/transmission_rate) * 1000 + queue_delay
print(f"hop_delay = {hop_delay}ms\n")

print("total_delay = hop_delay * num_routers+1")
print(f"{hop_delay} msec * {num_routers+1}")
print(f"total_delay = {hop_delay * (num_routers+1)} msec")


hop_delay = proc + trans + prop + queue
10 msec + 0.02 msec + 10.0 msec + 0 msec
hop_delay = 20.02ms

total_delay = hop_delay * num_routers+1
20.02 msec * 5
total_delay = 100.1 msec


In [4]:
# 计算超时间隔
EstimatedRTT = 100        # ms
DevRTT = 9                # ms
New_SampleRTT = 108       # ms
a = 0.125
b = 0.25

DevRTT = (1 - b) * DevRTT + b * abs(New_SampleRTT - EstimatedRTT)
EstimatedRTT = (1 - a) * EstimatedRTT + a * New_SampleRTT

TimeoutInterval = EstimatedRTT + 4 * DevRTT
print(TimeoutInterval, 'ms')

136.0 ms


In [5]:
# 求造成超时间隔不变的DevRTT
from sympy.solvers import solve
from sympy import Symbol

EstimatedRTT = 100        # ms
New_SampleRTT = 108       # ms
a = 0.125
b = 0.25
DevRTT = Symbol('DevRTT')

NewDevRTT = (1 - b) * DevRTT + b * abs(New_SampleRTT - EstimatedRTT)
NewEstimatedRTT = (1 - a) * EstimatedRTT + a * New_SampleRTT

f = EstimatedRTT + 4*DevRTT - NewEstimatedRTT - 4*NewDevRTT

result = solve(f, DevRTT)
print(result)


[9.00000000000000]


In [7]:
# Calculate the checksums (8 bits)
from functools import reduce

def overflowAdd(a, b):
    return ((a+b)&0xff) + 1 if (a+b)>>8 == 1 else (a+b)

myBits = ['01010101', '01110000', '01001100']
res = reduce(overflowAdd, map(lambda x: int(x,2), myBits))
print('1s complement: {0:08b}'.format(~res & 0xff))


# print('2s complement: {0:08b}'.format(-res & 0xff))

# Fancy way to do it
# print('1s complement: {0:08b}'.format(~reduce(lambda a, b: ((a+b)&0xff) + 1 if (a+b)>>8 == 1 else (a+b), map(lambda x: int(x,2), ['01010101', '01110000', '01001100'])) & 0xff))
# print('2s complement: {0:08b}'.format(-reduce(lambda a, b: ((a+b)&0xff) + 1 if (a+b)>>8 == 1 else (a+b), map(lambda x: int(x,2), ['01010101', '01110000', '01001100'])) & 0xff))


1s complement: 11101101


In [8]:
# Performance of rdt3.0 (stop-and-wait)

link = 10**9        # bits/sec  1 Gbps link
prop_delay = 15     # ms
packet_size = 8000  # bits

trans_delay = packet_size / link    # sec
print("time to transmit packet into channel:")
print(f'\tTransmission delay: {trans_delay*10**6} microsecs')
print(f"\tSender Utilization: 发送者忙于发送的时间的百分比 = {round((trans_delay*10**3)/(prop_delay*2+trans_delay*10**3), 5)}")

time to transmit packet into channel:
	Transmission delay: 8.0 microsecs
	Sender Utilization: 发送者忙于发送的时间的百分比 = 0.00027


In [9]:
# 计算窗口大小必须是多大才能使信道利用率大于某值

prop_delay = 15     # ms
rtt = (prop_delay*2)*(10**-3)    # sec
packets_len = 1500  # bytes
trans_link = 10**9  # bits/sec  1 Gbps link
channel_U = 0.98    # channel utilization 98%


x = ((packets_len*8)/trans_link) / (rtt+(packets_len*8)/trans_link)
print(f'{channel_U/x} packets')


2450.98 packets


In [10]:
import ipaddress

dest = {
    "Port 1": "128.8.16.0/20",
    "Port 2": "128.8.24.0/21",
    "Port 3": "128.8.128.0/24",
    "Port 4": "128.8.128.0/28",
}

def get_best_port(ip):
    ip = ipaddress.ip_address(ip)
    best_port = None
    best_net = None
    
    for port, net in dest.items():
        net = ipaddress.ip_network(net)
        if ip in net and (not best_net or net.prefixlen > best_net.prefixlen):
            best_port = port
            best_net = net
            
    return best_port
    
print(get_best_port("128.8.128.252"))
print(get_best_port("128.8.128.5"))
print(get_best_port("128.8.25.223"))
print(get_best_port("155.128.45.21"))

Port 3
Port 4
Port 2
None


In [11]:
ip = "12.1.0.0/17"

def getIPRange(ip):
    ip = ipaddress.ip_network(ip)
    return (ip[0], ip[-1])
    

print(getIPRange(ip))

(IPv4Address('12.1.0.0'), IPv4Address('12.1.127.255'))


In [12]:
def mask(number):
    return ipaddress.ip_address(2**32 - 2**(32-number))

print(mask(22))

255.255.252.0


In [14]:
def CRC_sender(D, G):
    D = D << G.bit_length() - 1
    while D.bit_length() >= G.bit_length():
        D ^= G << D.bit_length() - G.bit_length()
    return D

def CRC_receiver(D, G, R):
    D = D << (G.bit_length() - 1)
    D ^= R
    while D.bit_length() >= G.bit_length():
        D ^= G << D.bit_length() - G.bit_length()
    return D

G = 0b1101
D = 0b10101010
R = CRC_sender(D, G)

print("Sender: ", bin(R).replace('0b', ''))

print('Receiver: ', bin(CRC_receiver(D, G, R)).replace('0b', ''))
print('Receiver: ', bin(CRC_receiver(0b00101011, G, R)).replace('0b', ''))

Sender:  110
Receiver:  0
Receiver:  0
